## 학습 데이터 불러오기

In [11]:
import numpy as np
import json

data_in_path = './data_in/'
data_out_path = './data_out/'

input_train_data_file_name = 'train_input.npy'
label_train_data_file_name = 'train_label.npy'
data_configs_fine_name = 'data_configs.json'

input_data = np.load(open(data_in_path + input_train_data_file_name, 'rb'))
label_data = np.load(open(data_in_path + label_train_data_file_name, 'rb'))

prepro_configs = None

with open(data_in_path + data_configs_fine_name, 'r') as f:
    prepro_configs = json.load(f)

## 학습과 검증 데이터셋 분리

In [5]:
from sklearn.model_selection import train_test_split

random_seed = 13371447
test_split = 0.1

train_input, test_input, train_label, test_label = train_test_split(input_data, label_data, test_size=test_split, random_state=random_seed)
print(train_input.shape, train_label.shape)
print(test_input.shape, test_label.shape)

(22500, 174) (22500,)
(2500, 174) (2500,)


## 모델 구현

In [12]:
vocab_size = prepro_configs['vocab_size']

batch_size = 16
num_epochs = 3
word_embedding_dim = 100
hidden_state_dim = 150
dense_feature_dim = 150

learning_rate = 0.001

In [14]:
from tensorflow.keras import layers, Sequential

model = Sequential([
    layers.Embedding(vocab_size, word_embedding_dim),
    layers.Dropout(0.2),
    layers.LSTM(hidden_state_dim, activation='tanh', recurrent_activation='sigmoid'),
    layers.Dropout(0.2),
    layers.Dense(dense_feature_dim, activation='tanh'),
    layers.Dropout(0.2),
    layers.Dense(1, activation='sigmoid'),
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [16]:
history = model.fit(train_input, train_label, batch_size=batch_size, epochs=num_epochs, validation_data=(test_input, test_label))

Train on 22500 samples, validate on 2500 samples
Epoch 1/3
22500/22500 [==============================] - 208s 9ms/sample - loss: 0.6899 - accuracy: 0.5231 - val_loss: 0.6794 - val_accuracy: 0.5228
Epoch 2/3
22500/22500 [==============================] - 208s 9ms/sample - loss: 0.5929 - accuracy: 0.6657 - val_loss: 0.5923 - val_accuracy: 0.7092
Epoch 3/3
22500/22500 [==============================] - 209s 9ms/sample - loss: 0.5092 - accuracy: 0.7364 - val_loss: 0.3845 - val_accuracy: 0.8452


In [17]:
test_input_data = 'test_input.npy'

test_input = np.load(open(data_in_path + test_input_data, 'rb'))
test_input[0]

array([ 1723,     2,   165,  1262, 17596,  4164,  1752,  2769,   265,
        1580,  1051,   410,   825,   663,   963,   534,    88, 13095,
       32588,     2,   134,   239,   897,  5419,  1745,    25,   243,
        1694,  1341,  5222,  1190,   608,   245, 18038,  1577,    38,
        1256,  3793,    38,   987,  4866,    38, 12930,   440,    38,
        2605,  1723,  6128,   212,    11,  6850,     2,  1420,    20,
        2327,   279,  5260,  3985,   277,  1154,  4027,    67,  6532,
        5565,   941,  6468,   250,   748, 15838,  3771,  4422,   764,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,

In [18]:
predictions = model.predict(test_input)
predictions[:5]

InvalidArgumentError:  indices[22,72] = 74068 is not in [0, 74066)
	 [[node sequential_1/embedding_1/embedding_lookup (defined at <ipython-input-18-0b5732746803>:1) ]] [Op:__inference_distributed_function_19252]

Errors may have originated from an input operation.
Input Source operations connected to node sequential_1/embedding_1/embedding_lookup:
 sequential_1/embedding_1/embedding_lookup/18790 (defined at c:\dev\python37\lib\contextlib.py:112)

Function call stack:
distributed_function


In [ ]:
test_id_data = 'test_id.npy'